In [1]:
print("OK")

OK


In [8]:
%pwd

'd:\\Python\\MediBot\\MediBot\\research'

In [9]:
import os
os.chdir("../")

In [10]:
%pwd

'd:\\Python\\MediBot\\MediBot'

In [11]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [12]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents=loader.load()

    return documents

In [13]:
extracted_data = load_pdf_file(data='Data/')

In [9]:
#extracted_data

In [14]:
#Split the data into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [15]:
text_chunks = text_split(extracted_data)
print("Length", len(text_chunks))

Length 39994


In [16]:
from langchain.embeddings import HuggingFaceEmbeddings

In [17]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name=
                                       'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [18]:
embeddings = download_hugging_face_embeddings()

C:\Users\yeami\AppData\Local\Temp\ipykernel_30848\3950479710.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=


In [19]:
query_result = embeddings.embed_query("Hello MediBot")
print("Length", len(query_result))

Length 384


In [ ]:
#pcsk_2fLdn5_LjjsciubJjWxDXoaLn5jWVHXpkuWFQoS9BZCxfinrf86prSFYSxhrascEgKgHPZ

In [92]:
from dotenv import load_dotenv
load_dotenv()

True

In [93]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

In [94]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [5]:
import pinecone

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [24]:
from langchain_pinecone import PineconeVectorStore
index_name = "medibot"
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


Error while installing plugin inference: can't set attribute 'inference'
Traceback (most recent call last):
  File "c:\Users\yeami\anaconda3\envs\medibot\lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: can't set attribute 'inference'


In [25]:
#Existing index 
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

Error while installing plugin inference: can't set attribute 'inference'
Traceback (most recent call last):
  File "c:\Users\yeami\anaconda3\envs\medibot\lib\site-packages\pinecone_plugin_interface\actions\installation.py", line 13, in install_plugins
    setattr(target, plugin.namespace, impl(target.config, plugin_client_builder))
AttributeError: can't set attribute 'inference'


In [72]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs = {"k":3})

In [85]:
retrieved_docs = retriever.invoke("What is acne?")
retrieved_docs

[Document(id='cbf510d4-8959-4bbe-bc34-4740f26e2342', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\TGEM_V3_medibot.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='b3d402f0-a24c-4a44-9e17-385c4c950d2a', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'Data\\TGEM_V3_medibot.pdf', 'total_pages': 4505.0}, page_content='Sebaceous follicles— A structure found within the\nskin that houses the oil-producing glands and hair\nfollicles, where pimples form.\nSebum— An oily skin moisturizer produced by\nsebaceous glands.\nTretinoin— A drug that works by increasing the\ntur

In [96]:
###from langchain_openai import OpenAI
###llm = OpenAI(temperature=0.4, max_tokens=100)

from langchain_groq import ChatGroq
model: str ="deepseek-r1-distill-llama-70b"

deepmedibot = ChatGroq(api_key=GROQ_API_KEY, model_name= model)


In [97]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the"
    "answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [98]:
question_answer_chain = create_stuff_documents_chain(deepmedibot, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [107]:
response = rag_chain.invoke({"input": "What is blood pressure"})
print(response["answer"])

<think>
Okay, the user is asking, "What is blood pressure." I need to provide a concise answer using the given context. Let me read through the context again to extract the key points.

From the context, I see that blood pressure measures the force of blood against artery walls. It's recorded as two numbers: systolic and diastolic. Systolic is when the heart beats, pushing blood out, and diastolic is when the heart rests between beats.

I should explain it in simple terms, mentioning both systolic and diastolic pressures. Also, including an example like 110/70 mm Hg would make it clearer. I need to keep it within three sentences and make sure it's easy to understand.

So, putting it together: Blood pressure is the force of blood against artery walls, with systolic being the higher number during heartbeats and diastolic the lower when the heart rests. Normal readings are around 110/70 mm Hg, which can rise with age. That should cover the essentials without getting too technical.
</think

In [104]:
import site
print(site.getsitepackages())

['c:\\Users\\yeami\\anaconda3\\envs\\medibot', 'c:\\Users\\yeami\\anaconda3\\envs\\medibot\\lib\\site-packages']
